In [ ]:
import serial
import folium
from folium.plugins import MarkerCluster
from IPython.display import display, clear_output
import time
import numpy as np
import random

# Initialize map centered at a default location (e.g., New York City)
map_center = [12 + 50/60 + 41.6/3600, 77 + 39/60 + 51.0/3600]  # 12.84489, 77.66417
mymap = folium.Map(location=map_center, zoom_start=12)

# Create a marker cluster for better performance with many markers
marker_cluster = MarkerCluster().add_to(mymap)

# Function to update map with new GPS coordinates
def update_map(lat, lon):
    folium.Marker([lat, lon]).add_to(marker_cluster)
    mymap.save('gps_map.html')
    # Display the updated map in Jupyter Notebook
    clear_output(wait=True)
    display(mymap._repr_html_())

# Define quantum gates
X = np.array([[0, 1], [1, 0]])
I = np.array([[1, 0], [0, 1]])
Z = np.array([[1, 0], [0, -1]])
ZX = np.dot(Z, X)
H =  np.array([[1, 1], [1, -1]]) / np.sqrt(2)
CNOT = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

XI = np.kron(X, I)
II = np.kron(I, I)
ZI = np.kron(Z, I)
ZXI = np.kron(ZX, I)
HI = np.kron(H, I)

# Define 4x4 fault gates
X4 = np.kron(X, I)
Z4 = np.kron(Z, I)
H4 = np.kron(H, I)
I4 = np.kron(I, I)

# Function to randomly inject a fault
def inject_fault(init_state):
    # List of potential fault gates
    fault_gates = [X4, Z4, H4, I4]
    # Randomly select a fault gate
    fault_gate = random.choice(fault_gates)
    # Apply the selected fault gate to the initial state
    return np.dot(fault_gate, init_state)

# Function to encode GPS data
def encode_gps_data(lat, lon):
    binary_lat = ''.join(format(ord(x), '08b') for x in f"{lat:.6f}")
    binary_lon = ''.join(format(ord(x), '08b') for x in f"{lon:.6f}")
    msg = binary_lat + binary_lon
    encode = ''
    for i in range(0, len(msg) - 1, 2):
        bits = msg[i:i+2]
        if bits == '00':
            encode += 'I '
        elif bits == '11':
            encode += 'ZX '
        elif bits == '10':
            encode += 'Z '
        elif bits == '01':
            encode += 'X '
    return encode.strip()

# Function to process encoded message
def process_encoded_message(message):
    messageDigest = message.split(" ")

    digest = ""
    for m in messageDigest:
        init1 = np.array([1, 0, 0, 0])
        init2 = np.array([0, 0, 0, 1])
        if m == "I":
            init1 = np.dot(II, init1)
            init2 = np.dot(II, init2)
        elif m == "X":
            init1 = np.dot(XI, init1)
            init2 = np.dot(XI, init2)
        elif m == "Z":
            init1 = np.dot(ZI, init1)
            init2 = np.dot(ZI, init2)
        elif m == "ZX":
            init1 = np.dot(ZXI, init1)
            init2 = np.dot(ZXI, init2)
        else:
            print(f"Unknown Op: {m}")

        # Inject fault randomly with a 30% probability
        if random.random() < 0.3:  # 30% chance to inject a fault
            # Fault injection for the first state vector
            init1 = inject_fault(init1)
            # Fault injection for the second state vector
            init2 = inject_fault(init2)

        dig1 = np.dot(CNOT, init1)
        dig2 = np.dot(CNOT, init2)
        dig1 = np.dot(HI, dig1)
        dig2 = np.dot(HI, dig2)
        dig = dig1 + dig2
        if dig[0] != 0:
            digest += "00"
        elif dig[1] != 0:
            digest += "01"
        elif dig[2] != 0:
            digest += "10"
        elif dig[3] != 0:
            digest += "11"

    # Convert the binary digest to a string
    n = int(digest, 2)
    decoded_message = n.to_bytes((n.bit_length() + 7) // 8, 'big').decode('utf-8', errors='ignore')
    return decoded_message

# Try to initialize the serial connection with Arduino
try:
    ser = serial.Serial('COM3', 9600)  # Adjust 'COM3' based on your Arduino's port
except serial.SerialException as e:
    print(f"Could not open serial port: {e}")
    ser = None

if ser:
    # Main loop to continuously read GPS data and update map
    try:
        while True:
            if ser.in_waiting > 0:
                data = ser.readline().decode().strip()
                print(f"Received data: {data}")  # Debugging output
                if data.startswith("$GPGGA"):
                    gps_data = data.split(',')
                    if len(gps_data) >= 10 and gps_data[2] and gps_data[4]:  # Check if latitude and longitude fields are not empty
                        try:
                            # Extract latitude and longitude
                            lat_deg = int(gps_data[2][:2])
                            lat_min = float(gps_data[2][2:])
                            lat = lat_deg + (lat_min / 60.0)
                            if gps_data[3] == 'S':
                                lat = -lat

                            lon_deg = int(gps_data[4][:3])
                            lon_min = float(gps_data[4][3:])
                            lon = lon_deg + (lon_min / 60.0)
                            if gps_data[5] == 'W':
                                lon = -lon

                            print(f"Latitude: {lat}, Longitude: {lon}")

                            # Encode and process GPS data
                            encoded_message = encode_gps_data(lat, lon)
                            print(f"Encoded Message: {encoded_message}")

                            decoded_message = process_encoded_message(encoded_message)
                            print(f"Decoded Message: {decoded_message}")

                            # Update the map with decoded coordinates
                            # Assuming the decoded message is in the format "lat,lon"
                            decoded_lat, decoded_lon = map(float, decoded_message.split(','))
                            update_map(decoded_lat, decoded_lon)
                        except ValueError as e:
                            print(f"Error parsing GPS data: {e}")
                    else:
                        print("Incomplete GPS data received.")
            time.sleep(1)  # Adjust delay as needed to control the update frequency
    except KeyboardInterrupt:
        print("Mapping stopped by user.")
    finally:
        ser.close()
        print("Serial connection closed.")

    # Save the map to an HTML file one last time when the script ends
    mymap.save('gps_map.html')
    print("Map saved as 'gps_map.html'.")
else:
    print("Serial connection was not established. Exiting the program.")